In [ ]:
pip install python-dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import os
import sys
import tweepy 
import pickle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
uploaded = files.upload()

Saving .env to .env


In [ ]:
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [ ]:
Kenyan_Tweets = pd.read_csv('/content/drive/My Drive/Final/Edited Data/Key Data /KenyanTweets.csv', engine ='python')
Nigerian_Tweets = pd.read_csv('/content/drive/My Drive/Final/Edited Data/Key Data /NigerianTweets.csv', engine ='python')
South_African_Tweets = pd.read_csv('/content/drive/My Drive/Final/Edited Data/Key Data /SouthAfricanTweets.csv', engine ='python')

In [ ]:
Kenyan_Tweets.shape

(618574, 17)

In [ ]:
Nigerian_Tweets.shape

(1307255, 17)

In [ ]:
South_African_Tweets.shape

(633349, 17)

In [ ]:
Kenyan_Unique = Kenyan_Tweets.sort_values('original_author', ascending=False).\
drop_duplicates(subset='original_author').reset_index()

In [ ]:
Nigerian_Unique = Nigerian_Tweets.sort_values('original_author', ascending=False).\
drop_duplicates(subset='original_author').reset_index()

In [ ]:
South_African_Unique = South_African_Tweets.sort_values('original_author', ascending=False).\
drop_duplicates(subset='original_author').reset_index()

In [ ]:
Kenyan_Unique.shape

(632, 18)

In [ ]:
Nigerian_Unique.shape

(1370, 18)

In [ ]:
South_African_Unique.shape

(548, 18)

In [ ]:
Consumer_Key = os.environ.get('TWITTER_API_KEY')
Consumer_Secret = os.environ.get('TWITTER_API_SECRET')
Access_Token = os.environ.get('TWITTER_ACCESS_TOKEN')
Access_Token_Secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')


In [ ]:
# Create the authentication object
Authenticate = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret)

# Set the access token and access token secret
Authenticate.set_access_token(Access_Token, Access_Token_Secret)

# Create the API object while passing in the auth information
api = tweepy.API(Authenticate,wait_on_rate_limit=True)


In [ ]:
# define the class Clean and Transform
# This will be our custom transformer that will clean and return description
# custom transformer must have methods fit and transform
class CleanAndTransform(BaseEstimator):
    def __init__(self):
        self.stemmer = SnowballStemmer('english')
        #pass

    def fit(self, X:pd.DataFrame,y:pd.Series=None):
        return self

    def clean(self,string):
        letters_only = re.sub("[^a-zA-Z#@]", " ", string)
        words = letters_only.split()

        for i in range(0, len(words)):
            if "#" in words[i]:
                s = words[i].split('#')
                words[i] = '# '.join(s)
            if "@" in words[i]:
                s = words[i].split('@')
                words[i] = '@ '.join(s)
            if "http" in words[i]:
                s = words[i].split('http')
                words[i]= "http".join(s)

        total_stop_words = set(stopwords.words("english"))
        removed_stop_words = set(stopwords.words("english")[0:20])
        stop_words = total_stop_words - removed_stop_words
        content_words = [w for w in words if not w in stop_words]
        return " ".join(content_words)
    
    def stem(self, content_words):
        #take the content words and stem them
        words = content_words.split()
        roots = [self.stemmer.stem(word) for word in words]
        return " ".join(roots)

    def transform(self, X:pd.DataFrame):
        X=X.copy()
        X['description'] = X.apply(lambda row: self.clean(row['description']), axis = 1)
        X['description'] = X.apply(lambda row: self.stem(row['description']), axis = 1)
        x = X['description']
        return x
        

In [ ]:
#load the model
model = pickle.load(open("/content/drive/My Drive/Final/Models/User Category/model.pkl","rb"))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or inv

In [ ]:
Kenyan_User = pd.DataFrame(columns=['user_id','name','screen_name','description','followers_count', 'verified'],index=[i for i in range(len(Kenyan_Unique))])
i = 0
for index, row in Kenyan_Unique.iterrows():
  key = row['original_author']
  try:
    user = api.get_user(key)
  except:
    print("An exception occurred")
  
  Kenyan_User['user_id'][i] = user.id
  Kenyan_User['name'][i] = user.name
  Kenyan_User['screen_name'][i] = user.screen_name
  Kenyan_User['description'][i] = user.description
  Kenyan_User['followers_count'][i] = user.followers_count
  Kenyan_User['verified'][i] = user.verified
  i = i + 1

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [ ]:

Nigerian_User = pd.DataFrame(columns=['user_id','name','screen_name','description','followers_count', 'verified'],index=[i for i in range(len(Nigerian_Unique))])
i = 0
for index, row in Nigerian_Unique.iterrows():
  key = row['original_author']
  try:
    user = api.get_user(key)
  except:
    print("An exception occurred")
  
  Nigerian_User['user_id'][i] = user.id
  Nigerian_User['name'][i] = user.name
  Nigerian_User['screen_name'][i] = user.screen_name
  Nigerian_User['description'][i] = user.description
  Nigerian_User['followers_count'][i] = user.followers_count
  Nigerian_User['verified'][i] = user.verified
  i = i + 1

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [ ]:
South_African_User = pd.DataFrame(columns=['user_id','name','screen_name','description','followers_count', 'verified'],index=[i for i in range(len(South_African_Unique))])
i = 0
for index, row in South_African_Unique.iterrows():
  key = row['original_author']
  try:
    user = api.get_user(key)
  except:
    print("An exception occurred")
  
  South_African_User['user_id'][i] = user.id
  South_African_User['name'][i] = user.name
  South_African_User['screen_name'][i] = user.screen_name
  South_African_User['description'][i] = user.description
  South_African_User['followers_count'][i] = user.followers_count
  South_African_User['verified'][i] = user.verified
  i = i + 1

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [ ]:
Kenyan_User['group'] = model.predict(Kenyan_User[['description']])
Nigerian_User['group'] = model.predict(Nigerian_User[['description']])
South_African_User['group'] = model.predict(South_African_User[['description']])

In [ ]:
level_mapping = {0: 'general', 1: 'private', 2: 'NGO', 3:'government'}
Kenyan_User['group'] = Kenyan_User.apply(lambda row:Kenyan_User['group'].replace(level_mapping))
Nigerian_User['group'] = Nigerian_User.apply(lambda row:Nigerian_User['group'].replace(level_mapping))
South_African_User['group'] = South_African_User.apply(lambda row:South_African_User['group'].replace(level_mapping))

In [ ]:
Kenyan_Users= pd.merge(left=Kenyan_Tweets, right=Kenyan_User[['screen_name','group']], how='left', left_on='original_author', right_on='screen_name')
Nigerian_Users= pd.merge(left=Nigerian_Tweets, right=Nigerian_User[['screen_name','group']], how='left', left_on='original_author', right_on='screen_name')
South_African_Users= pd.merge(left=South_African_Tweets, right=South_African_User[['screen_name','group']], how='left', left_on='original_author', right_on='screen_name')

In [ ]:
Export_Kenyan_Users = Kenyan_Users[['original_author','group']]
Export_Kenyan_Users.to_csv("/content/drive/My Drive/Final/Edited Data/Key Data /Kenyan_User_Categories.csv")

In [ ]:
Export_Nigerian_Users = Nigerian_Users[['original_author','group']]
Export_Nigerian_Users.to_csv("/content/drive/My Drive/Final/Edited Data/Key Data /Nigerian_User_Categories.csv")

In [ ]:
Export_South_African_Users = South_African_Users[['original_author','group']]
Export_South_African_Users.to_csv("/content/drive/My Drive/Final/Edited Data/Key Data /South_African_User_Categories.csv")

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(20,7.5))
sns.countplot(Kenyan_Users['group'])
plt.xlabel("User Category")
plt.ylabel("Tweet Count")
plt.title("Kenyan Tweet Count per User Category")

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(20,7.5))
sns.countplot(Nigerian_Users['group'])
plt.xlabel("User Category")
plt.ylabel("Tweet Count")
plt.title("Nigerian Tweet Count per User Category")

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(20,7.5))
sns.countplot(South_African_Users['group'])
plt.xlabel("User Category")hh
plt.ylabel("Tweet Count")
plt.title("South African Tweet Count per User Category")